In [1]:
import folium

In [2]:
# let's start by creating an empty map using the IMO basemapbas

# basemap tile url and attribution
basemap_tiles = "https://geo.vedur.is/geoserver/www/imo_basemap_epsg3857/{z}/{x}/{y}.png"
basemap_attribution = "Icelandic Met Office | National Land Survey of Iceland | © OpenStreetMap contributors"

#create map and assign options
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)

#initiate and display map. You can also save as html with map.save("index.html")
map

In [3]:
# let's add a WMS layers and zoom in our map a bit
# we will use on of several glacier outlines available
# to get a full list of avaialble layers goto https://geo.vedur.is/geoserver/wms?request=GetCapabilities&service=WMS

map = folium.Map(location=[66.15, -22.2], zoom_start=11, tiles=basemap_tiles, attr=basemap_attribution)

# define parameters for a WMS layer
wms_overlay = folium.raster_layers.WmsTileLayer(
               url="https://geo.vedur.is/geoserver/wms", # url to wms server
               name="Glacier outlines around 1890", # name of wms layer for layer controls
               fmt='image/png',  # image/png is the default for format
               layers="glaciology:glacieroutline_lia", # name of wms layer
               transparent=True, # allow transparancy
               #opacity=0.5, # set transparancy
               overlay=True, # overlay wms layer on top of basemap
               )

wms_overlay.add_to(map)

map